# Project 3 - Constructing a database using API and Webscrapping

# Importing libraries

In [15]:
# Required Liberaries:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
from tqdm.auto import tqdm
import sqlalchemy as db

# Functions

In [75]:
def get_response(url):
    """
    Return a endpoint from a api, inputing a url and return html data.
    """
    response = requests.get(url, headers = headers)
    return response

def get_soup(url):
    """
    Entering a URL this function uses the BeautifulSoup library to return HTML content.
    """
    html = requests.get(url).content
    soup = BeautifulSoup(html)
    return soup

def get_souph(url):
    """
    Entering a URL, this function uses the BeautifulSoup library to return HTML content, 
    when the site has some access restriction and it is necessary to use the headers method 
    to get around this restriction.
    """
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    return soup

def get_dataframe(df: str, data, headers):
    """
    Transform a list of data and headers into a dataframe.
    """
    df = pd.DataFrame(np.array(data), columns = headers)
    return df

def create_table(title: str, df):
    """
    Create an engine, connect with PostgreSQL, create a new table and disconnect from the database.
    """
    engine = db.create_engine('postgresql://postgres:1fYS.9:f@localhost/games')
    conn = engine.connect()
    df.to_sql(title, con=conn, if_exists = 'replace', index=False)
    conn.close()

def update_table(title: str, df):
    """
    Create an engine, connect with PostgreSQL, update an existing table and disconnect from the database.
    """
    engine = db.create_engine('postgresql://postgres:1fYS.9:f@localhost/games')
    conn = engine.connect()
    df.to_sql(title, con=conn, if_exists='append', index = False)
    conn.close()

# USING API

![Twitch Logo](https://theroguewolfe.files.wordpress.com/2014/08/twitchlogo.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Twitch** is a video live streaming service operated by Twitch Interactive, a subsidiary of Amazon. Introduced in June 2011 as a spin-off of the general-interest streaming platform, Justin.tv, the site primarily focuses on video game live streaming, including broadcasts of eSports competitions, in addition to music broadcasts, creative content, and more recently, "in real life" streams. Content on the site can be viewed either live or via video on demand.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;I decided to use **Twitch Api** to search for data in this project, because reading its documentation it seemed simple to get access and its practical endpoints to work with. And the access is really intuitive! But the data is limited to some endpoint options, I tried to prioritize some information that can be used later relating to other data.

## Creating tables

### Games, Viewers and Channels

This table will show the games being played the most and their respective channel numbers.

In [17]:
# Tokens to get access into twitch api
headers = {'Client-ID' : 'h8zvs5dtl6ukn16cvf8s2ade7xw6km', 'Accept' : 'application/vnd.twitchtv.v5+json'}

In [18]:
#Empty dataframe
gameviewers = pd.DataFrame()

#Run a loop to get the top10 games
for i in range(0,11):
    
    #Getting Endpoint into each game to search for channels and viewers by the endpoint
    response = get_response('https://api.twitch.tv/kraken/games/top?offset='+str(i) +'&Limit=100')
    
    #List of Top Games
    games = response.json()['top']
    
    
    for game in games:
        #Inputing date
        today = datetime.date.today()
        
        #Inputing hour
        hour_list = datetime.datetime.now().strftime("%H:%M:%S")
        
        #Endpoint Data Gathering
        game_id = game['game']['_id']
        game_name = game['game']['name']
        viewers = int(game['viewers'])
        channels = int(game['channels'])
        
        #Creating a DataFrame
        mini_df = pd.DataFrame({'date':[today],
                                'hour': [hour_list],
                                'id':[game_id],
                               'name': [game_name],
                               'viewers': [viewers],
                               'channels': [channels]})
        gameviewers = pd.concat([gameviewers,mini_df])
gameviewers = gameviewers.reset_index(drop=True)

In [19]:
gameviewers

,date,hour,id,name,viewers,channels
0,2020-03-31,17:15:23,32399,Counter-Strike: Global Offensive,299484,4159
1,2020-03-31,17:15:23,509658,Just Chatting,285773,3896
2,2020-03-31,17:15:23,512710,Call of Duty: Modern Warfare,257112,10705
3,2020-03-31,17:15:23,21779,League of Legends,200665,6189
4,2020-03-31,17:15:23,32982,Grand Theft Auto V,169515,2898
...,...,...,...,...,...,...
101,2020-03-31,17:15:27,515448,Resident Evil 3,38496,277
102,2020-03-31,17:15:27,491931,Escape From Tarkov,36756,1545
103,2020-03-31,17:15:27,498566,Slots,29720,171
104,2020-03-31,17:15:27,512804,FIFA 20,26080,1356


### Most viewed channels of top games

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Using the endpoint of the previous table, we take the most watched games and search the channel endpoint to find out which channels are transmitting and receiving so many views.

In [20]:
#Empty DataFrame
stream_channel = pd.DataFrame()

#First loop to generate a list of top games being played
for i in tqdm(range(10)):
    
    #Topgames EndPoint
    response = get_response('https://api.twitch.tv/kraken/games/top')
    
    # Querying game
    games = response.json()
    top10games = (games['top'][i]['game']['name'])
    url_query = 'https://api.twitch.tv/kraken/search/streams?query='+top10games+'&Limit=100'
    response = requests.get(url_query, headers = headers)
    streams = response.json()['streams']
    
    #Second loop using streams list
    for i in range(10):
        
        try:
            today = datetime.date.today()
            hour_list = datetime.datetime.now().strftime("%H:%M:%S")
            stream_id = streams[i]['channel']['_id']
            stream_name = streams[i]['channel']['name']
            stream_game = streams[i]['game']
            stream_viewers = int(streams[i]['viewers'])
            stream_lang = streams[i]['channel']['broadcaster_language']
            stream_followers = int(streams[i]['channel']['followers'])
            stream_views = int(streams[i]['channel']['views'])
            
            #Inputing into a dataframe
            mini_df = pd.DataFrame({'date':[today],
                                    'hour':[hour_list],
                                    'streamer_id':[stream_id],
                                    'streamer_name': [stream_name],
                                    'game_name': [stream_game],
                                    'viewers': [stream_viewers],
                                    'language': [stream_lang],
                                    'followers': [stream_followers],
                                    'views': [stream_views]})
            stream_channel = pd.concat([stream_channel,mini_df])
        except:
            pass
stream_channel = stream_channel.reset_index(drop=True)

In [22]:
stream_channel = stream_channel.sort_values(by = 'viewers', ascending = False)
stream_channel.head(20).reset_index()

,index,date,hour,streamer_id,streamer_name,game_name,viewers,language,followers,views
0,0,2020-03-31,17:15:28,181077473,gaules,Counter-Strike: Global Offensive,68858,pt,1017253,100766175
1,1,2020-03-31,17:15:28,31239503,esl_csgo,Counter-Strike: Global Offensive,54877,en,3653190,450440157
2,20,2020-03-31,17:15:29,17337557,drdisrespect,Call of Duty: Modern Warfare,38627,en,4160843,168123578
3,21,2020-03-31,17:15:29,45044816,montanablack88,Call of Duty: Modern Warfare,33609,de,2368193,64494944
4,2,2020-03-31,17:15:28,490523982,flashpoint,Counter-Strike: Global Offensive,32676,en,69249,5439464
5,10,2020-03-31,17:15:28,39426641,evelone192,Just Chatting,28498,ru,452371,23926809
6,11,2020-03-31,17:15:28,50985620,papaplatte,Just Chatting,23851,de,560823,30497882
7,30,2020-03-31,17:15:29,83232866,ibai,League of Legends,22579,es,634317,16536718
8,22,2020-03-31,17:15:29,38244180,alanzoka,Call of Duty: Modern Warfare,21763,pt,2642397,95974633
9,12,2020-03-31,17:15:28,44445592,pokimane,Just Chatting,17765,en,4120683,99510438


# WEB SCRAPING

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;For Web Scraping I decide to search for data on **Steam** and **Metacrit**, they are sites that are reference in the area and contain information that may be useful in the future.

## Getting data from Steam

![Steam logo](http://icons.iconarchive.com/icons/martz90/circle/128/steam-icon.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Steam is a video game digital distribution service by Valve. It was launched as a standalone software client in September 2003 as a way for Valve to provide automatic updates for their games, and expanded to include games from third-party publishers. Steam has also expanded into an online web-based and mobile digital storefront. Steam offers digital rights management (DRM), matchmaking servers, video streaming, and social networking services. It also provides the user with installation and automatic updating of games, and community features such as friends lists and groups, cloud saving, and in-game voice and chat functionality.

### Getting the top 50 selled games on Steam

In [ ]:
def get_soup(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html)
    return soup

In [65]:
#Url to Scrapping
url = 'https://store.steampowered.com/search/?filter=topsellers'

#BeautifulSoup
soup = get_soup(url)

#List Comprehension to gathering data
mostselledgames = [games.text.strip().replace('\n','') for games in soup.find_all('div', attrs = {'class': 'col search_name ellipsis'})]
datereleased = [date.text for date in soup.find_all('div', attrs = {'class': 'col search_released responsive_secondrow'})]

#Using try and except to run the code despite some error
try:
    discount = [discount.text.strip().replace('-','').replace('%','') for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]
except:
    discount = [discount.text.strip() for discount in soup.find_all('div', attrs = {'class': 'col search_discount responsive_secondrow'})]

#Using try and except in case that game is free
try:
    price = [float(price.text.strip().split()[-1].replace(',','.')) for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]
except:
    price = [price.text.strip().split()[-1].replace(',','.') for price in soup.find_all('div', attrs = {'class': 'col search_price_discount_combined responsive_secondrow'})]

#Date column
date_list = [datetime.date.today() for i in range(len(mostselledgames))]

#Generating a DataFrame
data = list(zip(date_list, mostselledgames, datereleased, discount, price))
headers = ['date', 'game_name', 'date_released', 'discount', 'price R$']
mostselledgames = get_dataframe('mostselledgames', data, headers)

#### Most Selled Games DataFrame

In [56]:
mostselled_games

,date,game_name,date_released,discount,price R$
0,2020-03-31,Mount & Blade II: Bannerlord,"30 Mar, 2020",10,134.99
1,2020-03-31,RESIDENT EVIL 3,2020/04/03,,129.99
2,2020-03-31,Pummel Party,"20 Sep, 2018",34,19.13
3,2020-03-31,Terraria,"16 May, 2011",50,9.99
4,2020-03-31,The Witcher 3: Wild Hunt - Game of the Year Ed...,"30 Aug, 2016",70,29.99
5,2020-03-31,Grand Theft Auto V,"13 Apr, 2015",,69.99
6,2020-03-31,Euro Truck Simulator 2,"12 Oct, 2012",75,9.99
7,2020-03-31,Metro Exodus - Gold Edition,,55,55.79
8,2020-03-31,Monster Hunter World: Iceborne,"9 Jan, 2020",,89.99
9,2020-03-31,Risk of Rain 2,"28 Mar, 2019",20,44.79


### Getting a view of players online on steam by game

In [63]:
#Url to Scrapping
url = 'https://store.steampowered.com/stats/Steam-Game-and-Player-Statistics'

#BeautifulSoup
soup = get_soup(url)

#List Comprehension to gathering data
players_online = [players.text.strip().split()[0] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
daily_peak = [players.text.strip().split()[1] for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
game_name = [players.find('a').text for players in soup.find_all('tr', attrs = {'class': 'player_count_row'})]
date_list = [datetime.date.today() for i in range(len(game_name))]
hour_list = [datetime.datetime.now().strftime("%H:%M:%S") for i in range(len(game_name))]

#Generating a DataFrame
data = list(zip(date_list, hour_list, game_name, daily_peak, players_online))
headers = ['date_list', 'hour_list', 'game_name', 'daily_peak', 'players_online']
steam_playersonline = get_dataframe('steam_playersonline', data, headers)

#### Players Online by Games DataFrame

In [64]:
steam_playersonline

,date_list,hour_list,game_name,daily_peak,players_online
0,2020-03-31,20:22:13,Counter-Strike: Global Offensive,"1,152,459","645,141"
1,2020-03-31,20:22:13,Dota 2,"710,105","353,808"
2,2020-03-31,20:22:13,Mount & Blade II: Bannerlord,"219,787","177,749"
3,2020-03-31,20:22:13,Football Manager 2020,"164,398","100,767"
4,2020-03-31,20:22:13,Tom Clancy's Rainbow Six Siege,"175,527","99,888"
...,...,...,...,...,...
95,2020-03-31,20:22:13,Pummel Party,"9,602","5,588"
96,2020-03-31,20:22:13,Half-Life: Alyx,"6,873","5,480"
97,2020-03-31,20:22:13,Borderlands 2,"7,774","5,462"
98,2020-03-31,20:22:13,MORDHAU,"6,931","5,380"


## Getting data from Metacritic

![Metacritic Logo](https://upload.wikimedia.org/wikipedia/commons/thumb/4/48/Metacritic_logo.svg/726px-Metacritic_logo.svg.png)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic** is a website that aggregates reviews of films, TV shows, music albums, video games and formerly, books. The site provides an excerpt from each review and hyperlinks to its source. A color of green, yellow or red summarizes the critics' recommendations. It is regarded as the foremost online review aggregation site for the video game industry.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Metacritic's** scoring converts each review into a percentage, either mathematically from the mark given, or what the site decides subjectively from a qualitative review. Before being averaged, the scores are weighted according to the critic's fame, stature, and volume of reviews. The website won two Webby Awards for excellence as an aggregation website. Criticism has focused on the assessment system, alleged third-party attempts to influence the scores, and the lack of staff oversight of user reviews.

### Creating a table of games and their notes

In [67]:
url = 'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page=1'

In [77]:
#Starting a empty DataFrame
df = pd.DataFrame()

for i in range(2):
    #Headers to get around the system
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    #Url to Scrapping
    url = f'https://www.metacritic.com/browse/games/score/metascore/all/pc/filtered?page={i}'
    
    #BeautifulSoup
    html = requests.get(url, headers=headers).content
    soup = BeautifulSoup(html)
    
    #List Comprehension to gathering data
    game_name = [game.text.strip() for game in soup.find_all('div', attrs = {'class': 'basic_stat product_title'})]
    metacritic_score = [metascore.text for metascore in soup.find_all('div', attrs = {'class': 'metascore_w small game positive'})]
    user_score = [userscore.text.strip().replace('\n', '').split(':')[1] for userscore in soup.find_all('li', attrs = {'class': 'stat product_avguserscore'})]
    release_date = [release_date.text.strip().replace('\n', '') for release_date in soup.find_all('li', attrs = {'class': 'stat release_date full_release_date'})]
    
    #Generating a DataFrame
    data = list(zip(game_name, metacritic_score, user_score, release_date))
    headers = ['game_name', 'metacritic_score', 'user_score', 'release_date']
    metacrit = pd.DataFrame(np.array(data), columns=headers)

#### Games by Metacritic Score and Users Score

In [37]:
metacrit

,game_name,metacritic_score,user_score,release_date
0,Sid Meier's Civilization III,90,8.4,"Release Date:Oct 30, 2001"
1,Silent Hunter III,90,7.8,"Release Date:Mar 15, 2005"
2,Bayonetta,90,8.1,"Release Date:Apr 11, 2017"
3,Sid Meier's Civilization V,90,8.0,"Release Date:Sep 21, 2010"
4,Falcon 4.0: Allied Force,90,7.8,"Release Date:Jun 28, 2005"
...,...,...,...,...
95,Ori and the Blind Forest: Definitive Edition,88,8.8,"Release Date:Apr 27, 2016"
96,Shogo: Mobile Armor Division,88,8.4,"Release Date:Sep 30, 1998"
97,Thirty Flights of Loving,88,5.0,"Release Date:Aug 20, 2012"
98,Hearthstone: Heroes of Warcraft,88,5.9,"Release Date:Mar 11, 2014"


# CONNECTING PYTHON WITH POSTGRE

## Creating a function to connect and create a table

In [67]:
create_table(title = 'game_viewers', df = gameviewers)

In [68]:
create_table(title = 'top_channels', df = stream_channel)

In [69]:
create_table(title = 'steam_topselledgames', df = mostselled_games)

In [70]:
create_table(title = 'steam_playersonline', df = players_online)

In [40]:
create_table(title = 'metacrit_review', df = metacrit)

## Inserting New data to Tables

In [31]:
update_table(title = 'game_viewers', df = gameviewers)

In [32]:
update_table(title = 'top_channels', df = stream_channel)

In [33]:
update_table(title = 'steam_topselledgames', df = mostselled_games)

In [34]:
update_table(title = 'steam_playersonline', df = players_online)